In [1]:
sc = SparkContext.getOrCreate()

In [2]:
# Importação das bibliotecas que poderão ser utilizadas no algoritmo 

from PIL import Image
import timeit
import numpy as np
import pickle
import scipy.misc
import cv2
import random
import scipy.sparse as sps
from pyspark.mllib.linalg import Vectors

time_start = timeit.default_timer()

In [3]:
# Função Randomica de elementos para as matrizes b,g,r, com numero de 0 a 450 

def matriz(n_linhas, n_colunas):
    matriz = [] # Matriz
    linha = [] # Linha
    while len(matriz) != n_linhas: 
        n = random.randint(0,450) 
        linha.append(n) 
        if len(linha) == n_colunas:
            matriz.append(linha) 
            linha = []
    return matriz 

In [4]:
# Geração das Matrizes b,g,r 

matriz_lst = matriz (4,4)
b = np.array(matriz_lst)

matriz_lst = matriz (4,4)
g = np.array(matriz_lst)

matriz_lst = matriz (4,4)
r = np.array(matriz_lst)

linha = b.shape[0]
coluna = b.shape[1] 

# Kernel a ser utilizado no filtro de convolução 

kernel = np.array([[-2,-1,0],[-1,1,1],[0,1,2]])
if  sum(np.reshape(kernel,-1)) == 0:
    somaKernel = 1
else:  
    somaKernel= sum(np.reshape(kernel,-1))


In [5]:
# Módulo para apresentar conteúdos Gerados

print ('matriz b = ',b.shape)
print (b) # dimensões da matriz

print ('matriz g = ',g.shape)
print (g) # dimensões da matriz

print ('matriz r = ',r.shape)
print (r) # dimensões da matriz

print ("qtde linhas= ", linha)
print ("qtde colunas= ", coluna)

print (kernel)
print (somaKernel)

matriz b =  (4, 4)
[[ 53 170  39 153]
 [363 421 248 167]
 [279 400 413 266]
 [163 321 426 308]]
matriz g =  (4, 4)
[[ 95 235 380 446]
 [369 440 402 435]
 [252 213 381 167]
 [101 170 103 308]]
matriz r =  (4, 4)
[[425  77   5 433]
 [103  41 219 418]
 [417 142 207 316]
 [382 132 378 180]]
qtde linhas=  4
qtde colunas=  4
[[-2 -1  0]
 [-1  1  1]
 [ 0  1  2]]
1


In [6]:
# Criação do RDD concatenando as 3 matrizes (r,g,b) sendo o primeiro elemento a posição na matriz e um vetor 
# contendo o conteúdo do elemento nas matrizes b,g,r
# exemplo, na posição (0,0) criaremos um vetor onde o primeiro elemento corresponde ao valor da posição (0,0) na matriz b
# a sengunda corresponde à matriz g e a terceira r.

tuplasRDD= sc.parallelize((((i,j),[(b[i][j],g[i][j],r[i][j])]) for i in range(linha) for j in range(coluna)), 4)
indicesRDD = tuplasRDD.map(lambda g :g)

In [7]:
# Prints para avaliação do conteúdo 

print (type(indicesRDD))
print ("Qtde de Paertições = ", indicesRDD.getNumPartitions())
print (indicesRDD.collect())

<class 'pyspark.rdd.PipelinedRDD'>
Qtde de Paertições =  4
[((0, 0), [(53, 95, 425)]), ((0, 1), [(170, 235, 77)]), ((0, 2), [(39, 380, 5)]), ((0, 3), [(153, 446, 433)]), ((1, 0), [(363, 369, 103)]), ((1, 1), [(421, 440, 41)]), ((1, 2), [(248, 402, 219)]), ((1, 3), [(167, 435, 418)]), ((2, 0), [(279, 252, 417)]), ((2, 1), [(400, 213, 142)]), ((2, 2), [(413, 381, 207)]), ((2, 3), [(266, 167, 316)]), ((3, 0), [(163, 101, 382)]), ((3, 1), [(321, 170, 132)]), ((3, 2), [(426, 103, 378)]), ((3, 3), [(308, 308, 180)])]


In [8]:
# função que retona um vetor contendo pares ordenados onde a primeira tupla corresponde à submatriz onde existe o par (x,y)
# e a segunda tupla, a sua posição dentro da submatriz (x,y) 

#def submatrizes (x,y):
#                coordtransform = []
#                for i in range (0,3) :
#                    for j in range (0,3):
#                        dx = (x-i+1) 
#                        dy = (y-j+1)
#                        if (dx >= 0  and dy>=0 and (i-1)>=0 and (j-1)>=0 and dx < (linha-1) and dy < (coluna-1)):
#                            coordtransform.append (((i-1,j-1),((dx,dy))))
#                return coordtransform

In [9]:
# função a ser aplicada em indicesRDD
def funcIndices (x) : 
    coordtransform = []
    for x in range(linha):
        for y in range (coluna):
                indices = []
                for i in range (0,3) :
                    for j in range (0,3):
                        dx = (x-i+1) 
                        dy = (y-j+1)
                        if (dx >= 0  and dy>=0 and (i-1)>=0 and (j-1)>=0 and dx < (linha-1) and dy < (coluna-1)):
                               indices.append (((i-1,j-1),((dx,dy))))
                coordtransform.append (indices)
    return coordtransform

In [18]:
# cconteúdo gerado na aplicaççao da função 
print(funcIndices (0))

[[((0, 0), (0, 0))], [((0, 0), (0, 1)), ((0, 1), (0, 0))], [((0, 0), (0, 2)), ((0, 1), (0, 1))], [((0, 1), (0, 2))], [((0, 0), (1, 0)), ((1, 0), (0, 0))], [((0, 0), (1, 1)), ((0, 1), (1, 0)), ((1, 0), (0, 1)), ((1, 1), (0, 0))], [((0, 0), (1, 2)), ((0, 1), (1, 1)), ((1, 0), (0, 2)), ((1, 1), (0, 1))], [((0, 1), (1, 2)), ((1, 1), (0, 2))], [((0, 0), (2, 0)), ((1, 0), (1, 0))], [((0, 0), (2, 1)), ((0, 1), (2, 0)), ((1, 0), (1, 1)), ((1, 1), (1, 0))], [((0, 0), (2, 2)), ((0, 1), (2, 1)), ((1, 0), (1, 2)), ((1, 1), (1, 1))], [((0, 1), (2, 2)), ((1, 1), (1, 2))], [((1, 0), (2, 0))], [((1, 0), (2, 1)), ((1, 1), (2, 0))], [((1, 0), (2, 2)), ((1, 1), (2, 1))], [((1, 1), (2, 2))]]


In [ ]:
#o próximo passo seria concatenar cada elemento da lista gerada pela função indice com indicesRDD através de MAP.
#Para ilustrar, tomamos como exemplo a primeira posição de indices RDD (0.0),[53,95,425] , concatenariamos a 
#primeira posição da lista gerada pela função funcIndices [(0.0),(0.0)]

#após aplicarmos map , ficariamos com o primeiro elemento : [(0.0),[53,95,425],(((0.0),(0.0)),[53,95,425])] em indicesRDD e assim
#sucessivamente para todas as posições de indices RDD 